# r-3366-GUI

Taken on March 19th, Krypton run, Krypton trigger with "November 20162 conditions

In [1]:
import datetime

In [2]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2017-03-20 13:52:52


In [3]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [4]:
%gui qt
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [5]:
from invisible_cities.database import load_db
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.peak_functions_c as cpf
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.pmaps_functions as pmf

In [6]:
from invisible_cities.reco.pmap_io import pmap_writer, S12, S2Si

from   invisible_cities.core.configure import configure, print_configuration

from   invisible_cities.core.system_of_units_c import units
from   invisible_cities.cities.base_cities import PmapCity, SensorParams
from   invisible_cities.cities.base_cities import S12Params as S12P

In [7]:
from invisible_cities.core.core_functions import define_window
from invisible_cities.core.core_functions import lrange
from collections import namedtuple

In [81]:
from invisible_cities.reco.params import S12Params, ThresholdParams, CalibratedSum, PMaps, RawVectors,\
                                         SensorParams, CalibParams, PmapParams, CalibratedPMT, S1PMaps

In [9]:
%load_ext Cython

In [10]:
from matplotlib.figure import Figure

In [134]:
from event_maps import Main, pmt_calib_vectors, plot_pmt_waveforms, plot_pmt_signals_vs_time_mus, \
                       plot_signal_vs_time_mus, plot_s12, print_s12, plot_vector, plot_xy, scan_s12, hist_1d, \
                       compare_S1, features_s12, S12F

### Import GUI

In [12]:
main=Main()
main.show()

### Calibration vectors

In [13]:
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [14]:
P = pmt_calib_vectors()

In [15]:
P

CalibVectors(channel_id=array([ 0,  1,  4,  5,  8, 18, 22, 23, 26, 27, 31]), coeff_blr=array([ 0.00161,  0.00162,  0.00161,  0.00161,  0.00161,  0.0008 ,
        0.0008 ,  0.0008 ,  0.0008 ,  0.0008 ,  0.0016 ,  0.001  ]), coeff_c=array([  2.94000000e-06,   2.75000000e-06,   3.09000000e-06,
         2.81000000e-06,   2.88000000e-06,   1.00000000e-06,
         1.00000000e-06,   1.00000000e-06,   1.00000000e-06,
         1.00000000e-06,   2.76000000e-06,   1.00000000e-06]), adc_to_pes=array([ 25.17,  22.15,  33.57,  23.88,  21.55,  26.49,  25.39,  27.74,
        23.78,  20.83,  26.56,   0.  ]), pmt_active=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

## Files

In [16]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/3366/dst_waves_f0_3366.root.h5')

In [17]:
h5rwf = False

In [18]:
if h5rwf:
    h5rwf.close()

In [19]:
h5rwf = tb.open_file(RWF_file,'r')

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [20]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [21]:
pmt_active = list(range(11))

In [22]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))


        Number of events in file = 167
        Number of PMTs = 12
        PMTWL = 52000
        Number of SiPMs = 1792
        SiPMWL = 1300
      


### Params

In [198]:
s1par = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=6, lmax=20, stride=4, rebin=False)

### Maps

In [195]:
def event_maps(P, pmtrwf, sipmrwf, event=0):
    """ Events maps"""
    RWF = pmtrwf[event]
    CWF = blr.deconv_pmt(RWF, P.coeff_c, P.coeff_blr, n_baseline=48000, thr_trigger=5)
    CAL_PMT, CAL_PMT_MAU    =    cpf.calibrated_pmt_mau(CWF,
                                                        P.adc_to_pes,
                                                        pmt_active = P.pmt_active,
                                                        n_MAU = 100,
                                                        thr_MAU =  3)
    rms = [np.std(CAL_PMT[i]) for i in P.pmt_active]
    
    csum, csum_mau = cpf.calibrated_pmt_sum(CWF,P.adc_to_pes,P.pmt_active)
    s1par = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=6, lmax=20, stride=4, rebin=False)
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.3)
    S1 =  cpf.find_S12(s1_ene, s1_indx, **s1par._asdict())
    print('***S1****')
    print_s12(S1)
    
    for i in S1:
        print('S1 number {}'.format(i))
        s1f = features_s12(S1[i])
        print('S1 features = {}'.format(s1f))
        t = S1[i][0]
        tmin = t[0] - 100*units.ns
        tmax = t[-1] + 100*units.ns
        
        #print('S1 match region: [tmin:tmax] (mus) = {}:{}'.format(tmin/units.mus,tmax/units.mus))
        s1par_PMT = S12Params(tmin=tmin, tmax=tmax, lmin=3, lmax=20, stride=4, rebin=False)
        
        PMT_S1 = {}
        for pmt in P.pmt_active:
            s1_ene, s1_indx = cpf.wfzs(CAL_PMT_MAU[pmt], threshold=0.1)
            PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())
        nm = compare_S1(S1[i], PMT_S1)
        print('number of PMT matches = {}'.format(nm))
    
    s2_par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
    s2_ene, s2_indx = cpf.wfzs(csum, threshold=1.0)
    S2    = cpf.find_S12(s2_ene, s2_indx, **s2_par._asdict())
   
    print_s12(S2)
    if len(S2) == 0:
        return 0
    
    s2f = features_s12(S2[0])
    print('S2 features = {}'.format(s2f))
    t = S2[0][0]
    tmin = t[-1] + 1*units.mus
    print('S1p search starts at {}'.format(tmin))
    s1p_params = S12Params(tmin=tmin, tmax=1300*units.mus, stride=4, lmin=4, lmax=20, rebin=False)
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.1)
    S1p =  cpf.find_S12(s1_ene, s1_indx, **s1p_params._asdict())
    S12t, S12l, S12e = scan_s12(S1p)
    
    if(len(S1) == 1 and len(S2) == 1):
        dt = s2f.tpeak - s1f.tpeak
        
        print('drif time = {} mus'.format(dt/units.mus))
    
    print('***S2Si****')
    sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=3.5*units.pes, n_MAU=100)
    SIPM = cpf.select_sipm(sipm)
    S2Si = pf.sipm_s2_dict(SIPM, S2, thr=30*units.pes)

    
    main.add_figure('RWF', plot_pmt_waveforms(RWF, zoom=False, window_size=10000))
    main.add_figure('CWF_vs_time_mus',
                plot_pmt_signals_vs_time_mus(CWF,
                                             P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1200,
                                             signal_min =    -5,
                                             signal_max =  200))
    main.add_figure('Calibrated_PMTs', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 400,
                                             t_max      = 800,
                                             signal_min =  -2,
                                             signal_max =  10))
                    
    main.add_figure('PMT_rms',plot_xy(P.channel_id, rms))
    main.add_figure('Calibrated_SUM', 
                plot_signal_vs_time_mus(csum, t_min=0, t_max=1300, signal_min=-5, signal_max=60))
    main.add_figure('Calibrated_SUM_S1', 
                plot_signal_vs_time_mus(csum, t_min=0, t_max=640, signal_min=-2, signal_max=10))
    main.add_figure('Calibrated_SUM_S2', 
                plot_signal_vs_time_mus(csum, t_min=640, t_max=660, signal_min=-2, signal_max=100))
    main.add_figure('Calibrated_PMT_S1', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 0,
                                             t_max      = 640,
                                             signal_min =  -2,
                                             signal_max =  2))
    if len(S1) > 0:
        main.add_figure('S1', plot_s12(S1))
    if len(S2) > 0:
        main.add_figure('S2', plot_s12(S2))
    main.add_figure('S1p_t',hist_1d(S12t, xlo=650*units.mus, xhi=1300*units.mus))
    main.add_figure('S1p_l',hist_1d(S12l, xlo=0, xhi=20))
    main.add_figure('S1p_e',hist_1d(S12e, xlo=0, xhi=10))
    pmf.scan_s2si_map(S2Si)


In [197]:
event_maps(P, pmtrwf, sipmrwf,event=20)

***S1****
number of peaks = 1
S12 number = 0, samples = 11 sum in pes =6.518566533403748
time vector (mus) = [ 585.85   585.875  585.9    585.925  585.95   585.975  586.     586.025
  586.05   586.075  586.1  ]
energy vector (pes) = [ 0.40199683  0.86930846  0.99918481  0.95252814  0.72410708  0.34136451
  0.30223495  0.30669875  0.38500688  0.60882399  0.62731215]
S1 number 0
S1 features = S12F(tmin=585850.0, tmax=586100.0, tpeak=585900.0, w=250.0, etot=6.5185665334037477, emax=0.9991848050895642, er=0.15328290352937884)
number of PMT matches = 5
number of peaks = 1
S12 number = 0, samples = 7 sum in pes =6001.627505715247
time vector (mus) = [ 648.61375     649.6375      650.6375      651.6375      652.6375
  653.6525      654.58815789]
energy vector (pes) = [  118.82699895   689.38728416  1864.49332482  2190.81864371   929.84990157
   166.57143833    41.67991417]
S2 features = S12F(tmin=648613.75, tmax=654588.15789473685, tpeak=651637.5, w=5974.4078947368544, etot=6001.627505715247,

In [23]:
event=0
RWF = pmtrwf[event]

In [24]:
main.add_figure('RWF', plot_pmt_waveforms(RWF, zoom=False, window_size=10000))

In [25]:
CWF = blr.deconv_pmt(RWF, P.coeff_c, P.coeff_blr, n_baseline=48000, thr_trigger=5)

In [26]:
main.add_figure('CWF', plot_pmt_waveforms(CWF, zoom=False, window_size=10000))

In [27]:
main.add_figure('CWF_vs_time_mus',
                plot_pmt_signals_vs_time_mus(CWF,
                                             P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1200,
                                             signal_min =    -5,
                                             signal_max =  200))

In [28]:
CAL_PMT, CAL_PMT_MAU    =    cpf.calibrated_pmt_mau(
                               CWF,
                               P.adc_to_pes,
                               pmt_active = list(range(11)),
                               n_MAU = 100,
                               thr_MAU =  3)

In [29]:
rms = [np.std(CAL_PMT[i]) for i in P.pmt_active]

In [30]:
rms

[0.13925952503699188,
 0.18988275237228988,
 0.14793908358074045,
 0.13922641519407108,
 0.20270519939212719,
 0.23757858207682561,
 0.1964793708546671,
 0.19660480579001594,
 0.14619025685631598,
 0.16907160576246427,
 0.16005621327768194]

In [31]:
P.channel_id

array([ 0,  1,  4,  5,  8, 18, 22, 23, 26, 27, 31])

In [32]:
main.add_figure('Calibrated_PMTs', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 400,
                                             t_max      = 800,
                                             signal_min =  -2,
                                             signal_max =  10))

In [33]:
main.add_figure('PMT_rms',plot_xy(P.channel_id, rms))

In [34]:
csum, csum_mau = cpf.calibrated_pmt_sum(CWF,P.adc_to_pes,P.pmt_active)

In [35]:
main.add_figure('Calibrated_SUM', 
                plot_signal_vs_time_mus(csum, t_min=0, t_max=1300, signal_min=-5, signal_max=60))

In [36]:
main.add_figure('Calibrated_SUM_S1', 
                plot_signal_vs_time_mus(csum, t_min=550, t_max=600, signal_min=-2, signal_max=10))

In [37]:
main.add_figure('Calibrated_SUM_S2', 
                plot_signal_vs_time_mus(csum, t_min=640, t_max=660, signal_min=-2, signal_max=100))

In [38]:
main.add_figure('Calibrated_PMT_S1', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 560,
                                             t_max      = 570,
                                             signal_min =  -2,
                                             signal_max =  2))

In [39]:
main.add_figure('Calibrated_PMT_S2', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 640,
                                             t_max      = 660,
                                             signal_min =  -2,
                                             signal_max =  10))

In [40]:
s1par = S12Params(tmin=0*units.mus, tmax=620*units.mus, lmin=6, lmax=20, stride=4, rebin=False)

In [41]:
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.2)
S1 =  cpf.find_S12(s1_ene, s1_indx, **s1par._asdict())

In [42]:
print_s12(S1)

number of peaks = 1
S12 number = 0, samples = 8 sum in pes =4.184503416638211


In [43]:
main.add_figure('S1', plot_s12(S1))

In [44]:
S1[0][0]

array([ 564175.,  564200.,  564250.,  564275.,  564300.,  564325.,
        564350.,  564375.])

In [45]:
t = S1[0][0]

In [49]:
tmin = t[0] - 100*units.ns
tmax = t[-1] + 100*units.ns

In [50]:
s1par_PMT = S12Params(tmin=tmin, tmax=tmax, lmin=3, lmax=20, stride=4, rebin=False)

In [51]:
PMT_S1 = {}
    
for pmt in list(range(11)):
    s1_ene, s1_indx = cpf.wfzs(CAL_PMT_MAU[pmt], threshold=0.1)
    PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())

In [52]:
PMT_S1

{0: {0: [array([ 564275.,  564300.,  564325.,  564350.,  564375.]),
   array([ 0.1886195 ,  0.3082061 ,  0.42798349,  0.34914349,  0.27017628])]},
 1: {},
 2: {},
 3: {},
 4: {0: [array([ 564225.,  564250.,  564275.,  564300.]),
   array([ 0.19741919,  0.24417585,  0.24456594,  0.15207431])]},
 5: {0: [array([ 564250.,  564275.,  564300.,  564325.,  564350.]),
   array([ 0.14624002,  0.37283037,  0.44865761,  0.33571934,  0.18492569])]},
 6: {},
 7: {},
 8: {0: [array([ 564150.,  564175.,  564200.]),
   array([ 0.1503108 ,  0.27651703,  0.23466839])]},
 9: {},
 10: {}}

In [106]:
compare_S1(S1, PMT_S1)

4

In [55]:
s2_par = S12Params(tmin=620*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)

In [56]:
s2_ene, s2_indx = cpf.wfzs(csum, threshold=0.5)
S2              = cpf.find_S12(s2_ene, s2_indx, **s2_par._asdict())

In [58]:
print_s12(S2)

number of peaks = 1
S12 number = 0, samples = 8 sum in pes =5548.952226606702


In [59]:
main.add_figure('S2', plot_s12(S2))

In [60]:
t = S2[0][0]
tmin = t[-1] + 1*units.mus

In [61]:
s1p_params = S12Params(tmin=tmin, tmax=1300*units.mus, stride=4, lmin=4, lmax=20, rebin=False)

In [62]:
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.1)
S1p =  cpf.find_S12(s1_ene, s1_indx, **s1p_params._asdict())

In [72]:
S12t, S12l, S12e = scan_s12(S1p)

In [85]:
S12t/units.mus

array([  663.8  ,   675.6  ,   676.   ,   676.225,   729.825,   735.775,
         738.825,   739.425,   772.95 ,   790.125,   790.775,   791.225,
         825.525,   843.475,   844.125,   856.5  ,   856.825,   857.075,
         858.625,   860.55 ,   860.775,   879.175,   881.   ,   892.125,
         892.65 ,   893.425,   942.9  ,   951.1  ,  1017.45 ,  1018.875,
        1019.325,  1040.875,  1048.475,  1067.85 ,  1077.575,  1111.4  ,
        1120.775,  1121.475,  1172.775,  1184.425,  1184.875,  1210.625,
        1221.15 ,  1250.125,  1258.35 ,  1283.15 ])

In [90]:
S12l

array([ 4, 12,  5,  5,  4,  5,  5,  9,  5,  4,  6,  5,  4, 17,  7,  6,  5,
       11,  6,  4,  7,  5,  5, 11,  4,  5,  4,  5,  4, 14,  7,  4,  6,  4,
        4,  4, 13,  5,  5,  8, 15,  4,  4,  5,  5,  5])

In [93]:
S12e

array([ 1.33578137,  4.68475057,  1.83204878,  1.65008565,  0.99754446,
        1.38571219,  1.45342644,  4.83244309,  1.18557195,  1.24960416,
        4.85666601,  1.14894777,  1.16390972,  4.7269626 ,  4.13192381,
        2.60109528,  1.62381392,  5.17319124,  3.50308452,  0.75610212,
        4.2571463 ,  1.1207497 ,  1.64150105,  3.37867788,  1.13941775,
        1.4246488 ,  0.71109499,  0.95501715,  1.01405978,  4.66909534,
        1.61661456,  1.16445021,  1.49907679,  1.21008107,  1.03846749,
        0.75744174,  3.27173295,  1.78426881,  1.7920126 ,  2.31884286,
        6.62468596,  0.9835525 ,  1.20420064,  1.23241195,  1.63786143,
        1.55100131])

In [77]:
hist

array([ 4,  5,  6, 11,  2,  3,  4,  3,  5,  3])

In [78]:
bin_edges

array([  663.8  ,   725.735,   787.67 ,   849.605,   911.54 ,   973.475,
        1035.41 ,  1097.345,  1159.28 ,  1221.215,  1283.15 ])

In [88]:
main.add_figure('S1p_t',hist_1d(S12t/units.mus, xlo=S12t[0]/units.mus, xhi=S12t[-1]/units.mus))

In [92]:
main.add_figure('S1p_l',hist_1d(S12l, xlo=0, xhi=20))

In [94]:
main.add_figure('S1p_e',hist_1d(S12e, xlo=0, xhi=10))

In [97]:
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=3.5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=30*units.pes)


In [98]:
S2Si

{0: {1688: array([  4.46084186,  13.4669971 ,  14.46077975,   6.6347414 ,
           4.95773318,   0.        ,   0.        ,   0.        ]),
  1689: array([  4.11322481,  24.12610811,  24.68897045,  15.68317297,
           5.17640924,   0.        ,   0.        ,   0.        ])}}

In [99]:
S2

{0: [array([ 647248.75,  648287.5 ,  649287.5 ,  650287.5 ,  651287.5 ,
          652287.5 ,  653287.5 ,  654101.  ]),
  array([  104.85434639,   392.01175852,  1040.54204411,  1667.3030003 ,
          1376.86022385,   731.9631686 ,   193.76803026,    41.64965458])]}

In [101]:
pmf.scan_s2si_map(S2Si)


SiPM number = 1688, total energy = 43.98109328513487
SiPM number = 1689, total energy = 73.7878855667182


In [102]:
xs[1688]

195.0

In [103]:
xs[1689]

195.0

In [104]:
ys[1688]

-35.0

In [105]:
ys[1689]

-25.0